#MobileNet: Simple CNN model using in Mobile Webapplication,fast,: depth wise seperable convolution

load libraries

In [16]:
from flask import Flask, request, jsonify, url_for, render_template
import uuid #unique id so that it can be used as a identifier for predictions
import os #to get the current working directory


In [17]:
Allowed_extension = set(["png","jpg","jpeg","gif"])

In [18]:
def allowed_file(filename):
    return "." in filename and filename.rsplit(".")[1] in Allowed_extension

#create flask Application

In [19]:
app = Flask(__name__) #object:resource: html, css, js and other files

In [20]:
@app.route("/")
def index():
    return render_template("ImageML.html")

Create a model

In [21]:
import numpy as np
from werkzeug.utils import secure_filename

from tensorflow.keras.applications import MobileNet #CNN mobile net model
from PIL import Image, ImageFile
from io import BytesIO


In [22]:
#for mobile net it need some type of data some preprocessing input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.mobilenet import decode_predictions

In [23]:
model = MobileNet(weights = "imagenet", include_top = True)

In [24]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [25]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
IMAGE_CHANNEL = 3

In [26]:
@app.route("/api/image", methods=["POST"])
def upload_image():
    if "image" not in request.files:
        return render_template("ImageML.html", prediction="No Posted Image")
    file = request.files["image"] #else part
    
    if file.filename == "":
        return render_template("ImageML.html", prediction="You did not select image")
    
    #correctly solved then we say that file is successful to be uploaded
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        
        x = [] #numpy array containing pixels of image
        ImageFile.LOAD_TRUNCATED_IMAGES = False
        
        #load image here
        img = Image.open(BytesIO(file.read()))
        img.load()
        
        #This is all to be done for model
        img = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.ANTIALIAS)
        
        #numpy array
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        
        x = preprocess_input(x) #input is read
        
        pred = model.predict(x)
        lst = decode_predictions(pred, top = 3)
        
        
        items = []
        for item in lst[0]:
            items.append({"name":item[1], "prob":float(item[2])})
            
        response = {"pred":items}
        return render_template("ImageML.html", prediction=f"My Prediction: {response['pred']}")
    else:
        return render_template("ImageML.html", prediction="Invalid File Extension")

# running our code

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader = False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2023 19:50:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 19:50:46] "GET /static/css/style.css HTTP/1.1" 304 -


49152/35363 [=========================================] - 0s 1us/step


127.0.0.1 - - [01/May/2023 19:50:52] "POST /api/image HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 19:50:52] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 19:51:06] "POST /api/image HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 19:51:06] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 19:51:13] "POST /api/image HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 19:51:13] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 19:51:24] "POST /api/image HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 19:51:24] "GET /static/css/style.css HTTP/1.1" 304 -
